In [1]:
import pandas as pd
import networkx as nx
import json

In [2]:
data_path = 'data/tweets/'

lula_df = pd.read_csv(data_path+'2022-11-22-O LULA TÁ ELEITO.csv', index_col=0)
bols_df = pd.read_csv(data_path+'2022-11-22-#BolsonaroReeleito.csv', index_col=0)

print(f'''
O LULA TA ELEITO dataset size: {lula_df.shape[0]}
#BolosonaroReeleito dataset size: {bols_df.shape[0]}
''')


O LULA TA ELEITO dataset size: 13561
#BolosonaroReeleito dataset size: 37569



#### Mergind Datasets

In [3]:
merged_df = pd.concat([lula_df, bols_df])
merged_df.drop_duplicates(inplace=True)
merged_df.shape

(50843, 14)

In [4]:
merged_df.head(5)

created_at             tweet_id  \
0  2022-11-23 02:58:53  1595250504073973760   
1  2022-11-23 02:58:48  1595250484465618946   
2  2022-11-23 02:58:34  1595250425078456320   
3  2022-11-23 02:58:33  1595250420594933761   
4  2022-11-23 02:58:16  1595250350067441665   

                                       tweet_content            user  \
0  O minions já pediram mais 72 horas? \nKkk\nO L...    kaneda202210   
1  🚨Me sigam, vamos nos juntar, fora Bolsonaro!\n...   Alana47736058   
2  O LULA TÁ ELEITO\n\naquieta o cu aí, bolsomini...       qui_nunes   
3  Bora ganhar também o 4º turno dessas eleições....  Raquel35376102   
4  @DENISEF93287285 Podem chorar mas O LULA TÁ EL...    kaneda202210   

                                           user_info  has_mention  \
0  {'id': 1581370477641179138, 'name': 'Kaneda', ...        False   
1  {'id': 1236466004638740481, 'name': 'Nhana', '...        False   
2  {'id': 1297308695404175365, 'name': 'bartolome...        False   
3  {'id': 1318301646414295041, 'name': 'Raquel', ...        False   
4  {'id': 1581370477641179138, 'name': 'Kaneda', ...         True   

                                            mentions  is_reply reply_to  \
0                                                NaN     False      NaN   
1                                                NaN     False      NaN   
2                                                NaN     False      NaN   
3                                                NaN     False      NaN   
4  [{'id': '1483982093101244422', 'username': 'DE...     False      NaN   

   is_quote quoted_from  is_retweet  \
0     False         NaN        True   
1     False         NaN        True   
2     False         NaN       False   
3     False         NaN        True   
4     False         NaN        True   

                                      retweeted_from                hashtags  
0  {'user': '_Janoninho', 'user_id': 156389790744...                     NaN  
1  {'user': 'marcosaraujjoo', 'user_id': 27493482...  ['#Bolsonaroreeleito']  
2                                                NaN                     NaN  
3  {'user': '_Janoninho', 'user_id': 156389790744...                     NaN  
4  {'user': 'Elipsico', 'user_id': 60989020, 'twe...                     NaN

#### Saving data

In [5]:
users = list(set(merged_df['user'].tolist()))
tweets_content = merged_df['tweet_content'].tolist()
tweet_user_mapping = dict()

for user in users:
    tweet_user_mapping[user] = merged_df.query(f'user == "{user}"')['tweet_content'].tolist()

print(f'Number of users: {len(users)}')

Number of users: 20447


In [6]:
data = dict()
data['users'] = users
data['tweets'] = tweets_content
data['tweet_user_mapping'] = tweet_user_mapping

In [7]:
with open('data/tweets/data.json', 'w') as f:
    json.dump(data, f)

#### Creating retweet network

In [62]:
G = nx.DiGraph()

In [63]:
edges = list()

rt_df = merged_df.query('is_retweet == True')

for idx, row in rt_df.iterrows():
    #edges.append((row['user'], eval(row['retweeted_from'])['user'], 'retweet'))
    G.add_edge(row['user'], eval(row['retweeted_from'])['user'], interac_type='retweet')

# for idx, row in merged_df.iterrows():
#     if row['is_retweet'] and row['retweeted_from'] != 'NOT AVAILABLE':
#         edges.append((row['user'], eval(row['retweeted_from'])['user'], 'retweet'))
#     elif row['is_reply'] and row['reply_to'] != 'NOT AVAILABLE':
#         edges.append((row['user'],eval(row['reply_to'])['user'],'reply'))
#     elif row['is_quote'] and row['quoted_from'] != 'NOT AVAILABLE':
#         edges.append((row['user'],eval(row['quoted_from'])['user'],'quote'))

#print(f'Number of edges (with repetition): {len(edges)}')

In [57]:
#edges_df = pd.DataFrame(columns=['source', 'target', 'type'], data=edges)

In [58]:
# edges_count = edges_df.value_counts()
# edges_count

In [59]:
# for x in range(len(edges_count.values)):
#     edges_index = edges_count.index
#     edges_weights = edges_count.values
#     G.add_edge(edges_index[x][0], edges_index[x][1], interac_type=edges_index[x][2], weight=edges_weights[x])

In [64]:
list(G.edges(data=True))[:2]

[('kaneda202210', '_Janoninho', {'interac_type': 'retweet'}),
 ('kaneda202210', 'Elipsico', {'interac_type': 'retweet'})]

In [66]:
nx.info(G)

/tmp/ipykernel_37346/1064119803.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G)


'DiGraph with 18321 nodes and 37350 edges'

In [67]:
# nx.write_gexf(G, 'data/networks/social_graph.gexf')
nx.write_gexf(G, 'data/networks/rt_social_graph.gexf')